# Tsunami Vortex Analysis - Complete Guide

**Purpose**: Interactive guide to running tsunami vortex simulations using the MECH0020 framework.

This notebook demonstrates:
- How to set up and run simulations
- Standard mode (command-line) workflow
- Adaptive convergence agent for mesh refinement
- Output analysis and visualization

## 1. Environment Setup

### Requirements

- **MATLAB**: R2020b or later (no toolboxes required)
- **Python**: 3.7+ (for this notebook only)
- **Repository**: Clone from GitHub

### Path Setup

This notebook assumes you're running it from the repository root or `docs/03_NOTEBOOKS/` directory.
The cells below detect the repository root and set up paths accordingly.

In [ ]:
import os
import sys
from pathlib import Path

# Detect repository root
current_dir = Path.cwd()
if current_dir.name == '03_NOTEBOOKS':
    repo_root = current_dir.parent.parent
elif (current_dir / 'Scripts').exists():
    repo_root = current_dir
else:
    repo_root = current_dir  # Assume we're at root

print(f"Repository root: {repo_root}")
print(f"Scripts directory: {repo_root / 'Scripts'}")

# Verify key directories exist
assert (repo_root / 'Scripts' / 'Drivers').exists(), "Scripts/Drivers not found!"
assert (repo_root / 'Scripts' / 'Editable').exists(), "Scripts/Editable not found!"
print("✓ Repository structure verified")

## 2. Running Your First Simulation (Standard Mode)

### Method 1: From MATLAB Command Window

The simplest way to run a simulation is directly in MATLAB:

```matlab
% Open MATLAB and navigate to the repository
cd Scripts/Drivers
Analysis
```

When the startup dialog appears, select **"Standard Mode"**. The simulation will run with these defaults:
- Method: Finite Difference (FD)
- Mode: Evolution
- Initial Condition: Lamb-Oseen vortex
- Grid: 128×128
- Time: dt=0.001, Tfinal=1.0

**Expected runtime**: ~10-30 seconds (depending on your machine)

**Output location**: `Data/Output/FD/Evolution/FD_Evol_LambOseen_YYYYMMDD_HHMMSS/`

### Method 2: Programmatic Execution

For batch processing or custom configurations, you can call MATLAB from Python:

In [ ]:
import subprocess

# Example: Run a quick test simulation
# Note: This requires MATLAB to be installed and in your PATH
# Uncomment to run:

# matlab_script = f"""
# cd('{repo_root / 'Scripts' / 'Drivers'}');
# % Simulation would run here via Analysis.m
# % (Skipped in notebook to avoid modal dialogs)
# disp('MATLAB is available and paths are correct');
# exit;
# """
# 
# result = subprocess.run(
#     ['matlab', '-batch', matlab_script],
#     capture_output=True,
#     text=True
# )
# print(result.stdout)

print("To run simulations, use MATLAB directly as shown above.")
print("This notebook focuses on analyzing existing results.")

## 3. Configuration Files

All user-editable parameters are centralized in `Scripts/Editable/`:

### Default_FD_Parameters.m
Physics and numerical parameters:
```matlab
Parameters.nu = 0.001;           % Viscosity
Parameters.Lx = 2*pi;            % Domain size X
Parameters.Ly = 2*pi;            % Domain size Y  
Parameters.Nx = 128;             % Grid points X
Parameters.Ny = 128;             % Grid points Y
Parameters.dt = 0.001;           % Time step
Parameters.Tfinal = 1.0;         % Final time
Parameters.ic_type = 'Lamb-Oseen'; % Initial condition
```

### Default_Settings.m
Operational settings:
```matlab
Settings.save_figures = true;    % Save plots
Settings.save_data = true;       % Save MAT files
Settings.save_reports = true;    % Generate reports
Settings.monitor_enabled = true; % Live monitor
Settings.monitor_theme = 'dark'; % Monitor theme
Settings.append_to_master = true;% Master table
```

**To customize**: Edit these files directly, or override in your script:
```matlab
Parameters = Default_FD_Parameters();
Parameters.Nx = 256;  % Override defaults
```

## 4. UI Mode

For interactive use, launch UI Mode:

```matlab
cd Scripts/Drivers
Analysis
```

Select **"UI Mode"** from the startup dialog.

### UI Features

**Tab 1 - Configuration**:
- Method selection (FD, Spectral, Finite Volume)
- Mode selection (Evolution, Convergence, ParameterSweep, Plotting)
- Parameter editor with validation
- Initial condition designer with live preview

**Tab 2 - Live Monitor**:
- Real-time metrics (CFL, max vorticity, enstrophy, energy)
- Progress bar and ETA
- Dark theme console output
- Performance monitoring

**Tab 3 - Results**:
- Browse previous runs by ID
- Load and visualize saved results
- Export figures and data

For more details, see the [README](../../README.md#quick-start).

## 5. Adaptive Convergence Agent

The adaptive convergence agent performs intelligent mesh refinement without brute-force grid sweeps.

### What It Does

1. **Preflight testing**: Runs a few test cases to establish convergence patterns
2. **Pattern recognition**: Analyzes how metrics change with resolution
3. **Adaptive navigation**: Chooses next mesh size based on observed behavior
4. **Early stopping**: Stops when convergence tolerance is met
5. **Decision logging**: Saves complete trace of all iterations

### Running the Agent

In MATLAB:

```matlab
cd Scripts/Drivers
run_adaptive_convergence
```

**Expected output**:
```
ADAPTIVE CONVERGENCE AGENT - INTELLIGENT MESH REFINEMENT
========================================================================

Base Parameters:
  Domain: [10.0 x 10.0]
  Initial: Nx=64, Ny=64
  Time: dt=0.0100, Tfinal=1.00
  IC: stretched_gaussian
  Viscosity: 1.00e-06

Iteration 1: Testing Nx=64, Ny=64...
  Max vorticity: 1.234e-02
  Change from previous: N/A (first run)
  
Iteration 2: Testing Nx=96, Ny=96...
  Max vorticity: 1.235e-02  
  Change: 0.08% (converging)
  ...
  
CONVERGENCE ACHIEVED at Nx=128, Ny=128
Recommended configuration:
  Nx = 128
  Ny = 128
  dt = 0.0100
```

**Output files**:
- Convergence trace: `Data/Output/Convergence_Study/convergence_trace.csv`
- Preflight figures: `Data/Output/Convergence_Study/preflight/`

**Runtime**: Typically 5-15 iterations (1-5 minutes total for fast cases)

## 6. Analyzing Outputs

After running simulations, results are saved in `Data/Output/`:

### Directory Structure

```
Data/Output/
└── FD/
    └── Evolution/
        └── FD_Evol_LambOseen_20260207_143022/
            ├── Figures/      # PNG plots (300 DPI)
            │   ├── vorticity_contour_t0.00.png
            │   ├── streamlines_t0.00.png
            │   └── ...
            ├── Data/         # MAT files
            │   └── results.mat
            └── Reports/      # Text reports
                └── Report.txt
```

### Loading Results

To analyze results programmatically:

In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

# Example: Load results (you'll need to update path to an actual run)
# results_path = repo_root / 'Data' / 'Output' / 'FD' / 'Evolution' / 'FD_Evol_LambOseen_20260207_143022' / 'Data' / 'results.mat'

# if results_path.exists():
#     data = scipy.io.loadmat(results_path)
#     omega = data['omega']  # Vorticity field
#     X = data['X']          # X grid
#     Y = data['Y']          # Y grid
#     
#     plt.figure(figsize=(8, 6))
#     plt.contourf(X, Y, omega, levels=20, cmap='RdBu_r')
#     plt.colorbar(label='Vorticity')
#     plt.xlabel('X')
#     plt.ylabel('Y')
#     plt.title('Vorticity Field')
#     plt.show()
# else:
#     print("No results found. Run a simulation first!")

print("To analyze results:")
print("1. Run a simulation in MATLAB")
print("2. Note the output directory path")
print("3. Load the results.mat file from Data/ subdirectory")
print("4. Visualize using matplotlib or MATLAB")

### Report Files

Each run generates a professional report in `Reports/Report.txt`:

```
SIMULATION REPORT
=================

Run ID: FD_Evol_LambOseen_20260207_143022
Method: FD
Mode: Evolution
Initial Condition: Lamb-Oseen

PARAMETERS
----------
Grid: 128 x 128
Domain: 6.28 x 6.28
Time: dt=0.001, Tfinal=1.0
Viscosity: 0.001

RESULTS
-------
Wall Time: 12.34 s
Final Time: 1.0000
Max Vorticity: 1.234e-02
...
```

## 7. Next Steps

### Explore Different Modes

Try the other FD modes:

**Convergence Mode**:
```matlab
Run_Config = Build_Run_Config('FD', 'Convergence', 'Gaussian');
Parameters = Default_FD_Parameters();
Parameters.mesh_sizes = [32, 64, 128, 256];
Settings = Default_Settings();
[Results, paths] = ModeDispatcher(Run_Config, Parameters, Settings);
```

**ParameterSweep Mode**:
```matlab
Run_Config = Build_Run_Config('FD', 'ParameterSweep', 'Lamb-Oseen');
Parameters = Default_FD_Parameters();
Parameters.sweep_params = {'nu'};
Parameters.sweep_values = {[1e-5, 1e-4, 1e-3]};
Settings = Default_Settings();
[Results, paths] = ModeDispatcher(Run_Config, Parameters, Settings);
```

**Plotting Mode** (recreate plots from existing data):
```matlab
Run_Config = Build_Run_Config('FD', 'Plotting', 'Lamb-Oseen');
Settings = Default_Settings();
Settings.data_source = 'Data/Output/FD/Evolution/FD_Evol_LambOseen_20260207_143022/Data/results.mat';
[Results, paths] = ModeDispatcher(Run_Config, Parameters, Settings);
```

### Advanced Topics

- **Spectral methods**: Use `Spectral_Analysis.m` for pseudo-spectral solvers
- **Finite volume**: Use `Finite_Volume_Analysis.m` for FV discretizations
- **Variable bathymetry**: Use `Variable_Bathymetry_Analysis.m` for ocean floor topography
- **Custom initial conditions**: Edit `ic_factory.m` to add your own ICs

### Testing

Verify your installation:
```matlab
cd tests
Run_All_Tests
```

### Documentation

See additional documentation in `docs/`:
- [Repository Layout](../../docs/01_ARCHITECTURE/REPOSITORY_LAYOUT_AND_FILE_DOCUMENTATION.md)
- [OWL Framework](../../docs/01_ARCHITECTURE/OWL_Framework_Design.md)
- [UI Design](../../docs/02_DESIGN/UI_Research_And_Redesign_Plan.md)
- [README](../../README.md)